In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import keras
model = keras.models.load_model("/kaggle/input/numpy-in-txt-voices/voice-models")
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 2585, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 1292, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 1292, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 646, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 646, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 323, 128)     0

In [3]:

model.pop()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 2585, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 1292, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 1292, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 646, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 646, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 323, 128)     0

In [5]:
temp = ['/kaggle/input/numpy-in-txt-voices/data/data/Speaker0026/Speaker26_000.txt',
'/kaggle/input/numpy-in-txt-voices/data/data/Speaker0026/Speaker26_005.txt',
'/kaggle/input/numpy-in-txt-voices/data/data/Speaker0029/Speaker29_004.txt']

In [11]:
xx = load_data(temp,val=True)
xx = model.predict(xx)
xx.shape

1/1 [==============================] - 0s 29ms/step


(3, 128)

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(xx.shape[0]-1):
    
    t = xx[i]
    t = t[np.newaxis,:]
    
    for j in range(i,xx.shape[0]-1):
        tt = xx[j+1]
        tt = tt[np.newaxis,:]
        print(i,j+1,cosine_similarity(t,tt))

0 1 [[0.973052]]
0 2 [[-0.325133]]
1 2 [[-0.404859]]


In [7]:
from sklearn.preprocessing import OneHotEncoder
labels = os.listdir('/kaggle/input/numpy-in-txt-voices/data/data')

    

print(len((np.unique(labels))))
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(labels).reshape(-1, 1))



50


OneHotEncoder(handle_unknown='ignore')

In [4]:
finalids = []
startpath = '/kaggle/input/numpy-in-txt-voices/data/data'
for i in os.listdir(startpath):
    midpath = os.path.join(startpath,i)
    for j in os.listdir(midpath):
        finalpath = os.path.join(midpath,j)
        finalids.append(finalpath)
finalids[:5]

['/kaggle/input/numpy-in-txt-voices/data/data/Speaker0043/Speaker0043_018.txt',
 '/kaggle/input/numpy-in-txt-voices/data/data/Speaker0043/Speaker0043_028.txt',
 '/kaggle/input/numpy-in-txt-voices/data/data/Speaker0043/Speaker0043_012.txt',
 '/kaggle/input/numpy-in-txt-voices/data/data/Speaker0043/Speaker0043_005.txt',
 '/kaggle/input/numpy-in-txt-voices/data/data/Speaker0043/Speaker0043_025.txt']

In [5]:
from sklearn.model_selection import train_test_split

xtrain, ytrain = train_test_split(finalids,test_size=0.3,random_state=42)


In [6]:
len(xtrain),len(ytrain)

(1757, 754)

In [4]:
def load_data(ids,val =False):
    X = []
    Y = []

    for i in ids:
      # read one or more samples from your storage, do pre-processing, etc.
      # for example:
        x = np.loadtxt(i)
        x=x[:,:,np.newaxis]
        label = i.split('/')[-2]
       
        y = enc.transform(np.array(label).reshape(-1, 1)).toarray()
        
        
        X.append(x)
        Y.append(y[0])
    if val:
        return(np.array(X))
    else:
        return np.array(X), np.array(Y)

In [8]:
bat = 2
def batch_generator(ids, batch_size = bat,val =False):
    batch=[]
    while True:
            if not val:
                np.random.shuffle(ids) 
            for i in ids:
                #print(i)
                batch.append(i)
                if len(batch)==batch_size:
                    yield load_data(batch,val)
                    batch=[]

In [21]:

train_gen = batch_generator(xtrain,bat)
valdata_gen= batch_generator(ytrain,bat)
val_gen = batch_generator(ytrain,bat,val = True)

In [10]:
preds = model.predict(val_gen,steps = int(np.ceil(len(ytrain)/2)))



377/377 [==============================] - 263s 679ms/step


In [11]:
realval = []
for i in ytrain:
   
    realval.append(enc.transform(np.array(i.split('/')[-2]).reshape(-1, 1)).toarray()[0])
realval = np.array(realval)
realval.shape,realval[:2]

((754, 50),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]]))

In [14]:
metric = tf.keras.metrics.CategoricalAccuracy()
metric.update_state(realval,preds)
metric.result().numpy()

0.7546419

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
'''
model = keras.Sequential()
model.add(layers.Input((128,2585,1)))
model.add(layers.Conv2D(32, 3, activation='relu', padding="same"))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(64, 3, activation='relu', padding="same"))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(128, 3, activation='relu', padding="same"))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(256, 3, activation='relu', padding="same"))
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dense(256))
model.add(layers.Dense(128))
model.add(layers.Dense(50,activation = 'sigmoid'))
model.summary()'''

'\nmodel = keras.Sequential()\nmodel.add(layers.Input((128,2585,1)))\nmodel.add(layers.Conv2D(32, 3, activation=\'relu\', padding="same"))\nmodel.add(layers.MaxPool2D(2))\nmodel.add(layers.Conv2D(64, 3, activation=\'relu\', padding="same"))\nmodel.add(layers.MaxPool2D(2))\nmodel.add(layers.Conv2D(128, 3, activation=\'relu\', padding="same"))\nmodel.add(layers.MaxPool2D(2))\nmodel.add(layers.Conv2D(256, 3, activation=\'relu\', padding="same"))\nmodel.add(layers.GlobalMaxPooling2D())\nmodel.add(layers.Dense(256))\nmodel.add(layers.Dense(128))\nmodel.add(layers.Dense(50,activation = \'sigmoid\'))\nmodel.summary()'

In [121]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(train_gen,epochs=3, steps_per_epoch = int( np.ceil(len(xtrain) / bat)),validation_data = valdata_gen,validation_steps=int( np.ceil(len(ytrain) / bat) ))

Epoch 1/3
879/879 [==============================] - 933s 1s/step - loss: 0.3582 - accuracy: 0.8891 - val_loss: 0.3288 - val_accuracy: 0.8899
Epoch 2/3
879/879 [==============================] - 864s 984ms/step - loss: 0.3690 - accuracy: 0.8936 - val_loss: 0.4501 - val_accuracy: 0.8554
Epoch 3/3
879/879 [==============================] - 930s 1s/step - loss: 0.2267 - accuracy: 0.9300 - val_loss: 0.1379 - val_accuracy: 0.9403


In [30]:
model.save('voice-modelstest',save_format = "h5")

In [33]:
os.listdir('/kaggle/working')

['.virtual_documents',
 'voice-models',
 '__notebook_source__.ipynb',
 'voice-modelstest']

In [32]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/voice-modelstest.h5')

/kaggle/working/voice-modelstest.h5